In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fastai.transforms import TfmType
from fasterai.transforms import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.training import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(2)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [ ]:
IMAGENET = Path('data/imagenet/ILSVRC/Data/CLS-LOC/train')
proj_id = 'superres2x34_pre'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

gpath = IMAGENET.parent/(proj_id + '_gen_256.h5')

lr=5e-4
lrs = np.array([lr/100,lr/10,lr])

sz=128
scale=2
keep_pct=0.25
bs=64
lrs_unfreeze_factor=0.05
x_tfms = []
extra_aug_tfms = [RandomLighting(0.1, 0.1, tfm_y=TfmType.PIXEL)]
torch.backends.cudnn.benchmark=True

## Training

In [ ]:
netG = Unet34_V2(nf_factor=2, scale=scale).cuda()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

In [ ]:
dataLoader = ImageGenDataLoader(sz=sz, bs=bs, path=IMAGENET, keep_pct=keep_pct, x_tfms=x_tfms, reduce_x_scale=scale)

In [ ]:
md = dataLoader.get_model_data()

In [ ]:
wd=1e-7

In [ ]:
class LearnerGenModuleWrapper():
    def __init__(self, model: GeneratorModule, name:str):
        self.model = to_gpu(model)
        self.name = name

    def get_layer_groups(self, precompute):
        return self.model.get_layer_groups()

In [ ]:
unet = LearnerGenModuleWrapper(netG, 'unet')
learn = ConvLearner(md, unet)
imgGenVis = ImageGenVisualizationCallback(TENSORBOARD_PATH, netG, md, 'learner')
learn.metrics = []
learn.opt_fn=optim.Adam
learn.crit = F.mse_loss
#learn.crit = FeatureLoss(multiplier=1e2)

In [ ]:
learn.freeze_to(-1)

In [ ]:
#learn.lr_find(1e-4, 1e1, wds=wd, linear=False)

In [ ]:
#learn.sched.plot(n_skip=0, n_skip_end=0)

In [ ]:
learn.fit(lrs, 1, cycle_len=1, wds=wd, use_clr_beta=(10,10,0.95,0.85), callbacks=[imgGenVis])

In [ ]:
learn.freeze_to(0)

In [ ]:
learn.fit(lrs/10, 1, cycle_len=1, wds=wd, use_clr_beta=(10,10,0.95,0.85), callbacks=[imgGenVis])

In [ ]:
save_model(netG, 'superres2x34_gen_pretrain.h5')